In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import trange

import sys
sys.path.insert(0, '../src')

from megs.model.mPCA import mPCA
from megs.data import image, DataLoader

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle

# Load mm object from the file
filename = "morphmodel.pkl"
with open(filename, "rb") as file:
    mm = pickle.load(file)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
datamatrix = mm.datamatrix.reshape(mm.datamatrix.shape[0], 3, 64, 64)


data = mm.datamatrix.reshape(12484, 3, 64, 64)
data = data[:, 0, :]  # Use only the first map to learn the model


targets = mm.scores
# targets = datamatrix




# Split the data into train and test sets
train_data, test_data, train_targets, test_targets = train_test_split(data, targets, test_size=0.2, random_state=42)

# Assuming you have your training data and targets as tensors
train_data = torch.Tensor(train_data).to(device)
train_targets = torch.Tensor(train_targets).to(device)

test_data = torch.Tensor(test_data).to(device)
test_targets = torch.Tensor(test_targets).to(device)

# Convert test data and targets into a TensorDataset

# Convert training data and targets into a TensorDataset
train_dataset = TensorDataset(train_data, train_targets)
test_dataset = TensorDataset(test_data, test_targets)


# Define the batch size for training
batch_size = 128



# Create the train_loader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the CustomLayer and Autoencoder classes

class CustomLayer(nn.Module):
    def __init__(self, eigengalaxies, mean):
        super(CustomLayer, self).__init__()
        self.eigengalaxies = nn.Parameter(eigengalaxies, requires_grad=False)
        self.mean = mean

    def forward(self, x):
        # Compute the dot product along the PCA coefficients axis
        reconstructed = torch.tensordot(x, self.eigengalaxies.to(x.device), dims=1)
        reconstructed += self.mean.to(x.device)
        return reconstructed


class Autoencoder(nn.Module):
    def __init__(self, eigengalaxies, mean, pca_components=60):
        super(Autoencoder, self).__init__()
        self.mean = mean
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(32*16*16, pca_components),  # Adjust this according to the dimension after convolution
            nn.ReLU()
        )
        
        # Decoder
        self.decoder_stellar_age = nn.Sequential(
            CustomLayer(eigengalaxies[:, 0, :, :].to(device), mean[:, 0, :, :].to(device)),  # Assuming the 2nd dimension in eigengalaxies corresponds to the map type
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()  # outputs between 0 and 1
        )
        
        self.decoder_masses = nn.Sequential(
            CustomLayer(eigengalaxies[:, 1, :, :].to(device), mean[:, 1, :, :].to(device)),  # Assuming the 2nd dimension in eigengalaxies corresponds to the map type
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()  # outputs between 0 and 1
        )

    def forward(self, x):
        x = self.encoder(x)
        return self.decoder_stellar_age(x), self.decoder_masses(x)

# Set the eigengalaxies obtained from PCA and the mean
eigengalaxies = torch.tensor(eigengalaxies, dtype=torch.float32).to(device)
mean = torch.tensor(mean, dtype=torch.float32).to(device)

# Move the model to the GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
autoencoder = Autoencoder(eigengalaxies, mean).to(device)

# Define the loss function
criterion = nn.MSELoss()

# Define the optimizer
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

# Move the training and testing data to the GPU device
train_data = train_data.to(device)
test_data = test_data.to(device)

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader):
        optimizer.zero_grad()
        inputs = data.unsqueeze(1).to(device)  # Add a channel dimension for grayscale images
        outputs_stellar_age, outputs_masses = autoencoder(inputs)
        loss_stellar_age = criterion(outputs_stellar_age, inputs)
        loss_masses = criterion(outputs_masses, inputs)
        loss = loss_stellar_age + loss_masses
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")
    
# Prediction
with torch.no_grad():
    for data in test_loader:
        inputs = data.unsqueeze(1).to(device)  # Add a channel dimension for grayscale images
        outputs_stellar_age, outputs_masses = autoencoder(inputs)
        # Perform further processing or evaluation with the reconstructed maps